<a href="https://colab.research.google.com/github/mcash8/Info-Theory-Project/blob/main/CL%20Learning%20Experiments/Cascade_SaveActions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp drive/MyDrive/Cascade/training_parameters.py /content/
!cp drive/MyDrive/Cascade/Cascade_calculate_loaded_data_information1.py /content/
!cp drive/MyDrive/Cascade/kde.py /content/
!cp drive/MyDrive/Cascade/data_saving -r /content/

Imports 

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.io as sio
from keras.layers.core import Dense
import keras
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.models import Model
import tqdm
import time
import keras.backend as K
from time import sleep
import os
import pickle as cPickle
import matplotlib.pyplot as plt
from Cascade_calculate_loaded_data_information1 import calculate_layer_mutual_information,extract_probs,calculate_mutual_information,calculate_lower_mutual_information # Functions of calculating mutual information by different methods
from training_parameters import parameters # Import parametrs used for training model

Function Definitions 

In [ ]:
def load_generated_data(data_name):
    "load generated data"
    d=sio.loadmat(data_name)
    return d
def shuffle_in_unison_inplace(a, b):
    "Shuffle the dataset randomly"
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
def data_label_shuffle(data_sets_org, percent_of_train, min_test_data=80, shuffle_data=True):
    "Split the data to train and test"
    perc = lambda i, t: np.rint((i * t) / 100).astype(np.int32) # Calculate the index of train and test dataset according to percentage 
    C = type('type_C', (object,), {})
    data_sets = C()
    stop_train_index = perc(percent_of_train[0], data_sets_org['data_sets.data'].shape[0])
    start_test_index = stop_train_index
    if percent_of_train > min_test_data:
        start_test_index = perc(min_test_data, data_sets_org['data_sets.data'].shape[0])
    data_sets.train = C()
    data_sets.test = C()
    if shuffle_data:
        shuffled_data, shuffled_labels = shuffle_in_unison_inplace(data_sets_org['data_sets.data'], data_sets_org['data_sets.label'])
    else:
        shuffled_data, shuffled_labels = data_sets_org.data, data_sets_org.labels
    data_sets.train.data = shuffled_data[:stop_train_index, :]
    data_sets.train.labels = shuffled_labels[:stop_train_index, :]
    data_sets.test.data = shuffled_data[start_test_index:, :]
    data_sets.test.labels = shuffled_labels[start_test_index:, :]
    return data_sets
def store_file(stringOfHistory,his):
    "Store the history of accuarcy"
    with open(stringOfHistory,'wb') as fp:
        return cPickle.dump(his,fp)
def load_DATA(name):
    "Load saved history"
    with open(name, 'rb') as f:
        data=cPickle.load(f)
        return data

In [ ]:
class OutputObserver(keras.callbacks.Callback):
    'callback to observe the output of the network'
    def __init__(self, data_set,layernum,DATA,Label,bins,logs={}):
        self.out_log = dict()  # Save the output of each layer in each epoch
        self.out_log['eachlayer']=[]
        self.out_log['each_final_layer']=[]
        self.data_set = data_set
        self.layer_name='Layer'+str(layernum)
        self.DATA=DATA 
        self.Label=Label
        self.bins=bins
        self.local_IXT_cas_layer,self.local_ITY_cas_layer,self.finallocal_IXT_cas_layer,self.finallocal_ITY_cas_layer=[],[],[],[] # Save the information calculated by Binning method
        self.KDElocal_IXT_cas_layer,self.KDElocal_ITY_cas_layer,self.KDEfinallocal_IXT_cas_layer,self.KDEfinallocal_ITY_cas_layer=[],[],[],[] # Save the information calculated by PWD based upper bound method
        self.kdislocal_IXT_cas_layer,self.kdislocal_ITY_cas_layer,self.kdisfinallocal_IXT_cas_layer,self.kdisfinallocal_ITY_cas_layer=[],[],[],[]  # Save the information calculated by PWD based lower bound method
        self.pys1,self.unique_inverse_x,self. unique_inverse_y,self. pxs = extract_probs(self.Label,self.data_set) 
        self.PXs, self.PYs = np.asarray(self.pxs).T, np.asarray(self.pys1).T  # The distributation of Input and Label 
    def on_epoch_end(self, epoch, logs={}):
        "Calculate and save the information for each epoch"
        intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(self.layer_name).output) 
        layeroutput_cas=intermediate_layer_model.predict(self.data_set)                                           # Get the output of layer with name self.layer_name
        intermediate_predict_layer_model = Model(inputs=model.input,outputs=model.get_layer('Prediction_Layer').output) 
        finallayeroutput_cas=intermediate_predict_layer_model.predict(self.data_set)                              # Get the output of the classifier which is following the layer with name self.layer_name
        self.out_log['eachlayer'].append(layeroutput_cas)
        self.out_log['each_final_layer'].append(finallayeroutput_cas)
#        ==============================================================
        IX_CAS2,IY_CAS2=calculate_lower_mutual_information(layeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y) # Calculate information for cascaded layer by PWD based lower bound method
        finalIX_CAS2,finalIY_CAS2=calculate_lower_mutual_information(finallayeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y) # Calculate information for the classifier following the cascaded layer
        print('KdisI(X;T) is:'+str(IX_CAS2),'KdisI(Y;T) is:'+str(IY_CAS2))
        print('KdisI(X;T) is:'+str(finalIX_CAS2),'KdisI(Y;T) is:'+str(finalIY_CAS2))
        self.kdislocal_IXT_cas_layer.append(IX_CAS2)
        self.kdislocal_ITY_cas_layer.append(IY_CAS2)
        self.kdisfinallocal_IXT_cas_layer.append(finalIX_CAS2)
        self.kdisfinallocal_ITY_cas_layer.append(finalIY_CAS2)
#        ==============================================================
        IX_CAS,IY_CAS=calculate_mutual_information(layeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y) # Calculate information for cascaded layer  by PWD based upper bound method
        finalIX_CAS,finalIY_CAS=calculate_mutual_information(finallayeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y) # Calculate information for the classifier following the cascaded layer  by PWD based upper bound method
        print('KDEI(X;T) is:',IX_CAS,'KDEI(Y;T) is:',IY_CAS)
        print('KDEI(X;T) is:',finalIX_CAS,'KDEI(Y;T) is:',finalIY_CAS)
        self.KDElocal_IXT_cas_layer.append(IX_CAS)
        self.KDElocal_ITY_cas_layer.append(IY_CAS)
        self.KDEfinallocal_IXT_cas_layer.append(finalIX_CAS)
        self.KDEfinallocal_ITY_cas_layer.append(finalIY_CAS)
        
#        ================================================================================
        IX_CAS1,IY_CAS1=calculate_layer_mutual_information(layeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y,self.bins) # Calculate information for cascaded layer by Binning method
        finalIX_CAS1,finalIY_CAS1=calculate_layer_mutual_information(finallayeroutput_cas[0:len(self.DATA)],self.PXs, self.PYs,self.unique_inverse_x,self.unique_inverse_y,self.bins) # Calculate information for the classifier following the cascaded layer by Binning method
        self.local_IXT_cas_layer.append(IX_CAS1)
        self.local_ITY_cas_layer.append(IY_CAS1)
        self.finallocal_IXT_cas_layer.append(finalIX_CAS1)
        self.finallocal_ITY_cas_layer.append(finalIY_CAS1)
#        ==============================================================

Functions for creating network

In [ ]:
def connectOutputBlock(modelToConnectOut,outputsize,activation):
    "The final prediction block"
    modelToConnectOut.add(Dense(outputsize,activation=activation,name='Prediction_Layer'))
    return modelToConnectOut
def initialblock(structure_nodes,model,X_train,activation,layernum):
    "The initialization setting of the network"
    model.add(Dense(structure_nodes,activation=activation,input_dim=X_train.shape[1],kernel_regularizer=keras.regularizers.l1_l2(l1=0.01, l2=0.01),kernel_initializer=keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None),name='Layer'+str(layernum)))
    return model

def trainModel(modelToTrain,data,currentEpochs,batch_size,optimizer,accuarcy_name,filepath,layernum,DATA,Label):
    "The way of training model and save the relative results"
    modelToTrain.compile(loss='mean_squared_error',optimizer=optimizer,metrics=[accuarcy_name])
    bins = np.linspace(-1, 1, 25) # The intervals used in Binning method 
    bins = bins.astype(np.float32) 
    prediction_logs=OutputObserver(np.concatenate((data[0],data[2])),layernum,DATA,np.concatenate((data[1],data[3])),bins) # The function used to calculate mutual information for each epoch
    trainingResults = modelToTrain.fit(data[0],data[1], batch_size=batch_size, epochs=currentEpochs, validation_data=(data[2],data[3]),verbose=1,callbacks=[keras.callbacks.ModelCheckpoint(filepath+"/weights-improvement-{epoch:02d}.tf", monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1),prediction_logs])
    hist = trainingResults.history 
    # If you want to check the result by early stooping, add 'keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=nb_epoch/10, verbose=1, mode='auto')' into callbacks
    return hist,prediction_logs.out_log,prediction_logs.local_IXT_cas_layer,prediction_logs.local_ITY_cas_layer,prediction_logs.finallocal_IXT_cas_layer,prediction_logs.finallocal_ITY_cas_layer,prediction_logs

In [ ]:
#=====================================================================================================================
#                                          Start training process
#=====================================================================================================================

for number_of_data in [17]: # The number of generated datasets you are going to use, [17,28,44,48] are the datasets we used in paper
    for num_of_train in range(1): # The number of training times for get average information plane over multi times training
        lr,optimizer,batch_size,nb_epoch,activation,structure,num_of_experment,outputsize,percent_of_train,accuarcy_name,name_num_of_experment=parameters(number_of_data,num_of_train) # Load the parameters needed
        print(batch_size)
        Data_set=load_generated_data('data_saving/'+name_num_of_experment+'/dataset.mat') # Load datasets and label
        Data,Label=Data_set['data_sets.data'],Data_set['data_sets.label'] 
        DATA=Data # Rename data for further using and avoid covering
        Data_set_shuffled=data_label_shuffle(Data_set, percent_of_train,shuffle_data=True)
        X_train, Y_train,X_test, Y_test=Data_set_shuffled.train.data,Data_set_shuffled.train.labels,Data_set_shuffled.test.data,Data_set_shuffled.test.labels # Split train and test datasets
        #=============================================================================================================
        #                                        Cascade Training process
        #=============================================================================================================

#        ---------------------------------------------------
        print('TRAINING Cascade learning')
        history = dict() # Save the performance of each epoch
        output_history=dict() # Save the out put of each epoch and layer
        new_data_set=[X_train, Y_train,X_test, Y_test]
        # ==================================================================================
        sleep(0.1)
        #====================================================================training process==========================================================
        local_IXT_cas,local_ITY_cas,finallocal_IXT_cas, finallocal_ITY_cas=[],[],[],[] # Save information for each layer by Binning method
        KDElocal_IXT_cas, KDElocal_ITY_cas,KDEfinallocal_IXT_cas, KDEfinallocal_ITY_cas=[],[],[],[] # Save information for each layer by PWD upper bound method
        Kdislocal_IXT_cas, Kdislocal_ITY_cas,Kdisfinallocal_IXT_cas, Kdisfinallocal_ITY_cas=[],[],[],[] # Save information for each layer by PWD lower bound method
        for layernum in tqdm.tqdm(range(len(structure))): # Go through each layer cascaded, and tqdm used for showing progress bar 
              if layernum==0: # For the first layer, the input is the original datasets 
                  model = Sequential()
                  model=initialblock(structure[layernum],model,X_train,activation,layernum)
                  model = connectOutputBlock(model,outputsize,activation)
                  model.summary()
                  filepath='check_point_model_layer_save/model_'+num_of_experment+'/layer_'+str(layernum) # The path of saving model
                  if not os.path.exists(filepath):
                        os.makedirs(filepath)
                  history['iter'+str(layernum)] = []
                  output_history['iter'+str(layernum)] = []
                  hist,prediction_logs_out,local_IXT_cas_layer, local_ITY_cas_layer,finallocal_IXT_cas_layer, finallocal_ITY_cas_layer,prediction_logs=trainModel(model,new_data_set,nb_epoch,batch_size,optimizer,accuarcy_name,filepath,layernum,DATA,Label) # Get trained parameters and corresponding information 
                  history['iter'+str(layernum)].append(hist)
                  output_history['iter'+str(layernum)].append(prediction_logs_out)
                  X_train1,X_test1=X_train,X_test                                 # Copy the original data and label for avoiding cover and further using  
                  model_layer_path='model_layer_save/model_'+num_of_experment     # The path of saving model
                  if not os.path.exists(model_layer_path):                        # Create the file for saving model if this file is not existing
                      os.makedirs(model_layer_path)
                  model.save(model_layer_path+'/'+str(layernum)+'model.tf')
                  print('layer_'+str(layernum)+'_model is generated')
                  # =======================================================================================================================
                  #                        Save all mutual information given by different methods for first layer 
                  # =======================================================================================================================
                  local_IXT_cas.append(local_IXT_cas_layer)                       
                  local_ITY_cas.append(local_ITY_cas_layer)
                  finallocal_IXT_cas.append(finallocal_IXT_cas_layer)
                  finallocal_ITY_cas.append(finallocal_ITY_cas_layer)
                  # ========================================
                  KDElocal_IXT_cas.append(prediction_logs.KDElocal_IXT_cas_layer)
                  KDElocal_ITY_cas.append(prediction_logs.KDElocal_ITY_cas_layer)
                  KDEfinallocal_IXT_cas.append(prediction_logs.KDEfinallocal_IXT_cas_layer)
                  KDEfinallocal_ITY_cas.append(prediction_logs.KDEfinallocal_ITY_cas_layer)
                  # =============================================
                  Kdislocal_IXT_cas.append(prediction_logs.kdislocal_IXT_cas_layer)
                  Kdislocal_ITY_cas.append(prediction_logs.kdislocal_ITY_cas_layer)
                  Kdisfinallocal_IXT_cas.append(prediction_logs.kdisfinallocal_IXT_cas_layer)
                  Kdisfinallocal_ITY_cas.append(prediction_logs.kdisfinallocal_ITY_cas_layer)
                  # ===============================================================================================================================
              else: # Except first layer, the input is the output of previous layer
                  intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer('Layer'+str(layernum-1)).output) # Get the output of previous hidden layer, namely get the new input of the cascaded layer
                  prediction_class_train_updata=intermediate_layer_model.predict(X_train1) # Training part
                  prediction_class_test_updata=intermediate_layer_model.predict(X_test1)   # Testing part
                  new_data_set=[prediction_class_train_updata, Y_train,prediction_class_test_updata, Y_test] # Constructed dataset and label together, for convenient training
                  del model,intermediate_layer_model                                       # Delete previous model and constructed now cascaded model
                  model= Sequential()                                                      # Define the new model
                  model=initialblock(structure[layernum],model,prediction_class_train_updata,activation,layernum)
                  model = connectOutputBlock(model,outputsize,activation)
                  model.summary()
                  filepath='check_point_model_layer_save/model_'+num_of_experment+'/layer_'+str(layernum)                 # The path of saving model
                  if not os.path.exists(filepath):                                                                        # If the path is not existing, create it
                        os.makedirs(filepath)
                  history['iter'+str(layernum)] = []
                  output_history['iter'+str(layernum)] = []
                  hist,prediction_logs_out,local_IXT_cas_layer, local_ITY_cas_layer,finallocal_IXT_cas_layer,finallocal_ITY_cas_layer,prediction_logs=trainModel(model,new_data_set,nb_epoch,batch_size,optimizer,accuarcy_name,filepath,layernum,DATA,Label)#The training epoch can changed into nb_epoch*(layernum+1) as paper said
                  history['iter'+str(layernum)].append(hist)
                  output_history['iter'+str(layernum)].append(prediction_logs_out)
                  X_train1,X_test1=new_data_set[0],new_data_set[2]            # Copy the input of current cascaded layer for further using
                  model_layer_path='model_layer_save/model_'+num_of_experment # The path of saving model
                  if not os.path.exists(model_layer_path):                    # If the path is not existing, create it
                    os.makedirs(model_layer_path)
                  model.save(model_layer_path+'/'+str(layernum)+'model.tf')
                  print('layer_'+str(layernum)+'_model is generated')
                  # =======================================================================================================================
                  #                        Save all mutual information given by different methods for remaining layers
                  # =======================================================================================================================
#                  =========================================================
                  local_IXT_cas.append(local_IXT_cas_layer)
                  local_ITY_cas.append(local_ITY_cas_layer)
                  finallocal_IXT_cas.append(finallocal_IXT_cas_layer)
                  finallocal_ITY_cas.append(finallocal_ITY_cas_layer)
#                  ===============================================================
                  KDElocal_IXT_cas.append(prediction_logs.KDElocal_IXT_cas_layer)
                  KDElocal_ITY_cas.append(prediction_logs.KDElocal_ITY_cas_layer)
                  KDEfinallocal_IXT_cas.append(prediction_logs.KDEfinallocal_IXT_cas_layer)
                  KDEfinallocal_ITY_cas.append(prediction_logs.KDEfinallocal_ITY_cas_layer)
#                  =============================================
                  Kdislocal_IXT_cas.append(prediction_logs.kdislocal_IXT_cas_layer)
                  Kdislocal_ITY_cas.append(prediction_logs.kdislocal_ITY_cas_layer)
                  Kdisfinallocal_IXT_cas.append(prediction_logs.kdisfinallocal_IXT_cas_layer)
                  Kdisfinallocal_ITY_cas.append(prediction_logs.kdisfinallocal_ITY_cas_layer)
#                 ===============================================================================================================================
        "Save all the mutual information as files"
        information_save_path='cascade_information_save/'+num_of_experment  # The path of saving mutual information
        if not os.path.exists(information_save_path):
            os.makedirs(information_save_path)
        store_file(information_save_path+'/information_x.pkl',local_IXT_cas)
        store_file(information_save_path+'/information_y.pkl',local_ITY_cas)
        store_file(information_save_path+'/final_information_x.pkl',finallocal_IXT_cas)
        store_file(information_save_path+'/final_information_y.pkl',finallocal_ITY_cas)
#       =================================================================
        store_file(information_save_path+'/KDEinformation_x.pkl',KDElocal_IXT_cas)
        store_file(information_save_path+'/KDEinformation_y.pkl',KDElocal_ITY_cas)
        store_file(information_save_path+'/KDEfinal_information_x.pkl',KDEfinallocal_IXT_cas)
        store_file(information_save_path+'/KDEfinal_information_y.pkl',KDEfinallocal_ITY_cas)
#       ======================================================================
        store_file(information_save_path+'/Kdisinformation_x.pkl',Kdislocal_IXT_cas)
        store_file(information_save_path+'/Kdisinformation_y.pkl',Kdislocal_ITY_cas)
        store_file(information_save_path+'/Kdisfinal_information_x.pkl',Kdisfinallocal_IXT_cas)
        store_file(information_save_path+'/Kdisfinal_information_y.pkl',Kdisfinallocal_ITY_cas)
#       ==============================Save the performance===========================================
        accuarcy_save_path='history_accuarcy/acc_'+num_of_experment # The path of saving performance
        if not os.path.exists(accuarcy_save_path):
            os.makedirs(accuarcy_save_path)
        store_file(accuarcy_save_path+'/'+'acc.pkl',history)
        store_file(accuarcy_save_path+'/'+'output_history.pkl',output_history)
        print('history is saved')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


4096
TRAINING Cascade learning


  0%|          | 0/5 [00:00<?, ?it/s]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer0 (Dense)              (None, 10)                130       
                                                                 
 Prediction_Layer (Dense)    (None, 1)                 11        
                                                                 
Total params: 141
Trainable params: 141
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 0.4226 - acc: 0.6268
Epoch 1: saving model to check_point_model_layer_save/model_compression test[80.]_[10, 7, 5, 4, 3]_experment_17_0/layer_0/weights-improvement-01.tf


INFO:tensorflow:Assets written to: check_point_model_layer_save/model_compression test[80.]_[10, 7, 5, 4, 3]_experment_17_0/layer_0/weights-improvement-01.tf/assets


...

INFO:tensorflow:Assets written to: check_point_model_layer_save/model_compression test[80.]_[10, 7, 5, 4, 3]_experment_17_0/layer_4/weights-improvement-100.tf/assets


KdisI(X;T) is:4.720485712217325e-06 KdisI(Y;T) is:1.6304761470470178e-06
KdisI(X;T) is:6.61831966292839e-06 KdisI(Y;T) is:2.7450043415056096e-06
KDEI(X;T) is: 2.1910507292938863e-05 KDEI(Y;T) is: 8.661610615524187e-06
KDEI(X;T) is: 2.5877628724376513e-05 KDEI(Y;T) is: 9.132883387322079e-06
BINI(X;T) is:-0.0 BINI(Y;T) is:-0.0
BINI(X;T) is:-0.0 BINI(Y;T) is:-0.0
1/1 [==============================] - 7s 7s/step - loss: 0.2361 - acc: 0.6268 - val_loss: 0.2318 - val_acc: 0.6422


INFO:tensorflow:Assets written to: model_layer_save/model_compression test[80.]_[10, 7, 5, 4, 3]_experment_17_0/4model.tf/assets


INFO:tensorflow:Assets written to: model_layer_save/model_compression test[80.]_[10, 7, 5, 4, 3]_experment_17_0/4model.tf/assets
100%|██████████| 5/5 [56:36<00:00, 679.23s/it]

layer_4_model is generated
history is saved


#Plot

In [ ]:
!cp /content/model_layer_save -r drive/MyDrive/Cascade/tanh/
!cp /content/check_point_model_layer_save -r drive/MyDrive/Cascade/tanh/

In [ ]:
!cp drive/MyDrive/Cascade/tanh/ -r /content/model_layer_save
!cp  drive/MyDrive/Cascade/tanh/ -r /content/check_point_model_layer_save 

In [ ]:
!cp drive/MyDrive/Cascade/plot.py /content/

In [ ]:
from __future__ import print_function
import pickle as cPickle
import numpy as np
import plot as plo
import matplotlib.pyplot as plt
import os
from training_parameters import parameters # Import parametrs used for training model

In [ ]:
def load_DATA(name):
    "Load saved data"
    with open(name, 'rb') as f:
        data=cPickle.load(f)
        return data  
#---------------------figure setting--------------------------------------------
xticks = np.linspace(0,14,6)
yticks = np.linspace(0,1.2,5)
x_lim=-0.1 
y_lim=-0.1
axis_font=28
fig_size = (16,8)   

def plot_layer_sepreate(local_IXT_CAS,local_ITY_CAS,finallocal_IXT_CAS,finallocal_ITY_CAS,structure,num_of_experment):
        "Plot information separatly for each layer and corresponding classifier"
        fig1=plt.figure(figsize=fig_size)
        local_IXT_CAS_array,local_ITY_CAS_array=np.array(local_IXT_CAS),np.array(local_ITY_CAS)
        finallocal_IXT_CAS_array,finallocal_ITY_CAS_array=np.array(finallocal_IXT_CAS),np.array(finallocal_ITY_CAS)
        for layernum in range(len(structure)):
            ax=fig1.add_subplot(2,np.ceil(len(structure)/2),layernum+1)
            ax.scatter(local_IXT_CAS_array[layernum],local_ITY_CAS_array[layernum],label='CAS_layer'+str(layernum))
            print('the epoch is:'+str(len(local_IXT_CAS_array[layernum])))
            ax.scatter(local_IXT_CAS_array[layernum][-1],local_ITY_CAS_array[layernum][-1],label='CAS_layer'+str(layernum)+'final_epoch',facecolors='none', edgecolors='b')
            ax.scatter(finallocal_IXT_CAS_array[layernum],finallocal_ITY_CAS_array[layernum],label='finalCAS_layer'+str(layernum)) 
            ax.scatter(finallocal_IXT_CAS_array[layernum][-1],finallocal_ITY_CAS_array[layernum][-1],label='finalCAS_layer'+str(layernum)+'final_epoch',facecolors='none', edgecolors='g') 
            fig1.suptitle('information_comparison_'+num_of_experment,fontsize=axis_font)
            ax.set_xticks(xticks)
            ax.set_yticks(yticks)
            ax.set_xlim(x_lim)
            ax.set_ylim(y_lim)
            plt.xlabel('$I(X;T)$',fontsize=axis_font)
            plt.ylabel('$I(Y;T)$',fontsize=axis_font)
            plt.legend()
        # plt.show() # Show the picture after ploting
        information_figure_path='Cascade_information_result_saving/'+num_of_experment
        finalinformation_figure_path='Cascade_final_information_result_saving/'+num_of_experment
        if not os.path.exists(information_figure_path):
            os.makedirs(information_figure_path)
        if not os.path.exists(finalinformation_figure_path):
            os.makedirs(finalinformation_figure_path)
        fig1.savefig(information_figure_path+'/'+'hidden_final_compare.png')

In [ ]:
for number_of_data in [17]:
    for num_of_train in range(1):
        _,_,_,nb_epoch,_,structure,num_of_experment,outputsize,percent_of_train,accuarcy_name,name_num_of_experment=parameters(number_of_data,num_of_train)
        ###===========================plot training perfprmance========================
        accuarcy_save_path='history_accuarcy/acc_'+num_of_experment
        HISTORY=load_DATA(accuarcy_save_path+'/acc.pkl')
        max_epoch=np.array([len(HISTORY['iter'+str(layernum)][0][accuarcy_name]) for layernum in range(len(structure))]).max()
        print('The maximium epoch is:',str(max_epoch))
        epochsInds=np.linspace(1,max_epoch,max_epoch)
        hISTORY=dict()
        for layernum in range(len(structure)):
            hISTORY['iter_TRAIN'+str(layernum)]=[]
            hISTORY['iter_TEST'+str(layernum)]=[]
            for i in range(len(HISTORY['iter'+str(layernum)])):
                  hISTORY['iter_TRAIN'+str(layernum)].append(HISTORY['iter'+str(layernum)][i][accuarcy_name]) 
                  hISTORY['iter_TEST'+str(layernum)].append(HISTORY['iter'+str(layernum)][i]['val_'+accuarcy_name])
        #--------------------------------------------------------------------------------------------------------------------------------------
        accuarcy_image_save_path='image_history_accuarcy/acc_'+num_of_experment
        if not os.path.exists(accuarcy_image_save_path):
            os.makedirs(accuarcy_image_save_path)
        fig_all=plt.figure(figsize=(16,10))
        axes=fig_all.add_subplot(1,2,1)
        for layernum in range(len(structure)):
            axes.plot(hISTORY['iter_TRAIN'+str(layernum)][0],label='Layer '+str(layernum)+', Average:'+str(round(np.mean(hISTORY['iter_TRAIN'+str(layernum)][0][-100:-1]),3)))           
        plt.legend(fontsize=axis_font-10,loc=9,ncol=1,bbox_to_anchor=(0.62,0.77))
        # plt.grid(True)
        axes.set_xlim(-0.1)
        axes.set_ylim(0.6)
        axes.set_yticks(np.linspace(0.6,1,5)) 
        plt.xlabel('Epoch\n (a) Training accuarcy',fontsize=axis_font)
        plt.ylabel('Accuarcy',fontsize=axis_font)
        plt.xticks(rotation=45,fontsize=22)
        plt.yticks(fontsize=22)
        # ===========================================Plot embedding figure over first 50 epochs================================================================
        rect = [0.36,0.12,0.6,0.4]
        box = axes.get_position()
        width = box.width
        height = box.height
        inax_position  = axes.transAxes.transform(rect[0:2])
        transFigure = fig_all.transFigure.inverted()
        infig_position = transFigure.transform(inax_position)    
        xl = infig_position[0]
        yl = infig_position[1]
        width *= rect[2]
        height *= rect[3]  # <= Typo was here
        ax2= fig_all.add_axes([xl,yl,width,height])
        for layernum in range(len(structure)):
            ax2.plot(hISTORY['iter_TRAIN'+str(layernum)][0][0:50],label='Layer '+str(layernum))
        plt.xlabel('Epoch',fontsize=axis_font-2)
        ax2.set_xlim(0.0)
        plt.ylabel('Accuarcy',fontsize=axis_font-2)
        plt.xticks(fontsize=22)
        plt.yticks(fontsize=22)
        plt.legend(fontsize=axis_font-10)
        # plt.show()
        # ===========================================================================================================
        axes1=fig_all.add_subplot(1,2,2) 
        for layernum in range(len(structure)):
            axes1.plot(hISTORY['iter_TEST'+str(layernum)][0],label='Layer '+str(layernum)+', Average:'+str(round(np.mean(hISTORY['iter_TEST'+str(layernum)][0][-100:-1]),3)))
        plt.legend(fontsize=axis_font-10,loc=1,ncol=1,bbox_to_anchor=(0.985,0.77))
        # plt.grid(True)
        axes1.set_xlim(-0.1)
        axes1.set_ylim(0.6)
        axes1.set_yticks(np.linspace(0.6,1,5))
        # plt.title('ACC_TEST',fontsize=axis_font+2)
        plt.xlabel('Epoch\n (b) Testing accuarcy',fontsize=axis_font)
        plt.ylabel('Accuarcy',fontsize=axis_font)
        plt.xticks(rotation=45,fontsize=22)
        plt.yticks(fontsize=22)
        # plt.suptitle('all in one'+num_of_experment)
        # ==========================Plot embedding figure over first 50 epochs==============================================
        rect = [0.36,0.12,0.6,0.4]
        box = axes1.get_position()
        width = box.width
        height = box.height
        inax_position  = axes1.transAxes.transform(rect[0:2])
        transFigure = fig_all.transFigure.inverted()
        infig_position = transFigure.transform(inax_position)    
        xl = infig_position[0]
        yl = infig_position[1]
        width *= rect[2]
        height *= rect[3]  # <= Typo was here
        ax1= fig_all.add_axes([xl,yl,width,height])
        for layernum in range(len(structure)):
            ax1.plot(hISTORY['iter_TEST'+str(layernum)][0][0:50],label='Layer '+str(layernum))
        plt.xlabel('Epoch',fontsize=axis_font-2)
        ax1.set_xlim(0.0)
        plt.ylabel('Accuarcy',fontsize=axis_font-2)
        plt.xticks(fontsize=22)
        plt.yticks(fontsize=22)
        plt.legend(fontsize=axis_font-10)
        plt.show()
        fig_all.savefig(accuarcy_image_save_path+'/accuarcy.png',bbox_inches="tight")
        fig_all.savefig('image_history_accuarcy/accuarcy'+num_of_experment+'.png',bbox_inches="tight")
        ##==========================cas================================================
        information_save_path='cascade_information_save/'+num_of_experment
        local_IXT_CAS=load_DATA(information_save_path+'/information_x.pkl')
        local_ITY_CAS=load_DATA(information_save_path+'/information_y.pkl')
        finallocal_IXT_CAS=load_DATA(information_save_path+'/final_information_x.pkl')
        finallocal_ITY_CAS=load_DATA(information_save_path+'/final_information_y.pkl')
        #==============================plot final============================================
        information_figure_path='Cascade_information_result_saving/'+num_of_experment
        finalinformation_figure_path='Cascade_final_information_result_saving/'+num_of_experment
        if not os.path.exists(information_figure_path):
            os.makedirs(information_figure_path) 
        if not os.path.exists(finalinformation_figure_path):
            os.makedirs(finalinformation_figure_path)
        plot_layer_sepreate(local_IXT_CAS,local_ITY_CAS,finallocal_IXT_CAS,finallocal_ITY_CAS,structure,num_of_experment)
        local_IXT_CAS_array,local_ITY_CAS_array=np.array(local_IXT_CAS),np.array(local_ITY_CAS)
        finallocal_IXT_CAS_array,finallocal_ITY_CAS_array=np.array(finallocal_IXT_CAS),np.array(finallocal_ITY_CAS)
        print('Starting colorful map........')
        plo.plotfigure(local_IXT_CAS_array,local_ITY_CAS_array,epochsInds,information_figure_path)
        print('Starting final information')
        plo.plotfigure(finallocal_IXT_CAS_array,finallocal_ITY_CAS_array,epochsInds,finalinformation_figure_path)
        #del local_IXT_CAS,local_ITY_CAS,finallocal_IXT_CAS,finallocal_ITY_CAS,information_figure_path,finalinformation_figure_path,local_IXT_CAS_array,local_ITY_CAS_array,finallocal_IXT_CAS_array,finallocal_ITY_CAS_array
        

In [ ]:
sm = plt.cm.ScalarMappable(cmap='gnuplot', norm=plt.Normalize(vmin=0, vmax=100))
sm._A = []
PLOT_LAYERS    = None 
num_layers = 5

activations = ['tanh', 'relu']

if PLOT_LAYERS is None:
        PLOT_LAYERS = []
        for lndx in range(num_layers):
            #if d['data']['activity_tst'][lndx].shape[1] < 200 and lndx != num_layers - 1:
            PLOT_LAYERS.append(lndx)
fig=plt.figure(figsize=(5,5))


    
for epoch in range(1, epochsInds.shape[0]):
    c = sm.to_rgba(epoch)
    xmvals = local_IXT_CAS_array[:,epoch]
    ymvals = local_ITY_CAS_array[:, epoch]
    plt.plot(xmvals, ymvals, c=c, alpha=0.1, zorder=1)
    plt.scatter(xmvals, ymvals, s=20, facecolors=[c for _ in PLOT_LAYERS], edgecolor='none', zorder=2)


plt.ylim([0, 1])
plt.xlim([0, 12])
#     plt.ylim([0, 3.5])
#     plt.xlim([0, 14])
plt.xlabel('I(X;M)')
plt.ylabel('I(Y;M)')
plt.title('tanh')
    
cbaxes = fig.add_axes([1.0, 0.125, 0.03, 0.8]) 
plt.colorbar(sm, label='Epoch', cax=cbaxes)

DO_SAVE = True

!mkdir /content/plots
if DO_SAVE:
    plt.savefig('plots/' + 'Synthetic tanh batch')
!cp /content/plots -r drive/MyDrive/Cascade/tanh